In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('usedCar_Dataset.csv')

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

----------------------------------------------

##### 刪除重複值

In [4]:
the_vin = data[data.duplicated(subset='VIN', keep=False)]
the_vin[the_vin.duplicated(subset=['CarID'], keep=False)]
new_data = data[~data.duplicated(subset=['CarID'], keep='last')]
new_data.duplicated(subset=['CarID'], keep=False).sum()

0

In [5]:
the_vin = new_data[new_data.duplicated(subset=['VIN'], keep=False)]
new_data = new_data.drop(index=[1296653,1524990])

------------------------------------------------------

In [6]:
new_data['Certified'].unique()
new_data['Certified'].fillna(value=0, inplace=True)

In [7]:
def toint(x):
    if x == True:
        return int(x)
    else:
        return x

In [8]:
new_data['Certified'] = new_data['Certified'].apply(toint)

In [9]:
new_data['Certified'].unique()

array([0, 1], dtype=int64)

In [10]:
new_data[['Price','Certified']].corr()

,Price,Certified
Price,1.000000,0.130795
Certified,0.130795,1.000000


In [11]:
new_data = new_data.drop(columns=['CarID','VIN'])

In [12]:
new_data.head()

,MakeId,Title,BodyStyle,Rental,Location,GasMileage_city,GasMileage_highway,Price,Mileage,Transmission,FuelType,ExteriorColor,InteriorColor,DoorsNum,MaximumSeating,Engine,Drivetrain,OptionCount,MajorOptions,Certified,Accident Check,OwnershipHistory,SellingDays,OriginPrice
0,m1,"2007 Chevrolet Tahoe - $11,999",suv,NaN,"Garland, TX",NaN,NaN,11999.0,129280.0,Automatic,Gasoline,Gold,NaN,NaN,NaN,V8,NaN,0,NaN,0,0.0,2.0,1873,12999.0
1,m1,"2005 Chevrolet Express - $11,995",van,NaN,"Claremont, NH",NaN,NaN,11995.0,118113.0,Automatic,NaN,White,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,1.0,1863,NaN
2,m1,2010 Chevrolet Silverado 2500HD Work Truck Ext...,pickup_truck,NaN,"Frankfort, KY",NaN,NaN,15900.0,144000.0,Automatic,Gasoline,White,Gray,4 doors,6 seats,V8,Four-Wheel Drive,2,"Steel Wheels,Tow Package",0,0.0,2.0,1837,16900.0
3,m1,2014 Chevrolet Corvette Stingray 3LT Coupe RWD...,coupe,NaN,"Lewistown, MT",17 MPG City,29 MPG Highway,49995.0,2500.0,7-Speed Manual,Gasoline,Red,Black,2 doors,2 seats,V8,Rear-Wheel Drive,5,"Bluetooth,Backup Camera,Aluminum Wheels,Naviga...",0,0.0,1.0,1832,62900.0
4,m1,"2002 Chevrolet Monte Carlo SS FWD - $3,440",coupe,NaN,"Delavan, IL",17 MPG City,27 MPG Highway,3440.0,174000.0,Automatic,Gasoline,Black,Black,2 doors,5 seats,V6,Front-Wheel Drive,2,"Leather Seats,Aluminum Wheels",0,0.0,2.0,1809,4800.0


In [13]:
columns = new_data.columns

In [14]:
# 上面做了一些簡單的基本處理 , 詳細參考 20200121_fill_empty 這本筆記本
# 下面要補空值

----------------------------------------------------------------------

##### 補BodyStyle

In [15]:
# 用 每一個車款 各自 出現最多次的 BodyStyle 來補 BodyStyle

In [16]:
new_data1 = pd.DataFrame(columns=columns)
# 建一個空的 dataframe

In [17]:
# type(new_data1)

In [18]:
for id in new_data['MakeId'].unique():
    part = new_data[new_data['MakeId'] == id]
    most_frequent = part.groupby(by='BodyStyle').size().sort_values(ascending=False).index[0]
    part['BodyStyle'] = part['BodyStyle'].fillna(value=most_frequent)
    new_data1 = new_data1.append(part)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [19]:
new_data1.isnull().sum()

MakeId                      0
Title                       0
BodyStyle                   0
Rental                1968035
Location                    0
GasMileage_city        467051
GasMileage_highway     467051
Price                  127920
Mileage                 35969
Transmission            43611
FuelType                55844
ExteriorColor           57359
InteriorColor          413991
DoorsNum              1137476
MaximumSeating         114967
Engine                  60655
Drivetrain             115055
OptionCount                 0
MajorOptions           310481
Certified                   0
Accident Check          32305
OwnershipHistory        70501
SellingDays                 0
OriginPrice           1122205
dtype: int64

---------------------------------------

##### 補 Rental

In [20]:
# Rental 的空值補法是 : 由於 Rental 和 Mileage有關
# 假如 該筆資料的 Mileage 大於 第三四分位數 就讓 Rental = True

In [21]:
new_data2 = pd.DataFrame(columns=columns)

In [22]:
def fillRental(x, third_quantile):
    if x.Rental != True:
        if x.Mileage > third_quantile:
            return True
        else:
            return False
    else:
        return True

In [23]:
for id in new_data1['MakeId'].unique():
    part = new_data1[new_data1['MakeId'] == id]
    third_quantile = part.Mileage.quantile(0.75)
    part['Rental'] = part.apply(lambda x:fillRental(x, third_quantile), axis=1)
    new_data2 = new_data2.append(part)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [24]:
new_data2.isnull().sum()

MakeId                      0
Title                       0
BodyStyle                   0
Rental                      0
Location                    0
GasMileage_city        467051
GasMileage_highway     467051
Price                  127920
Mileage                 35969
Transmission            43611
FuelType                55844
ExteriorColor           57359
InteriorColor          413991
DoorsNum              1137476
MaximumSeating         114967
Engine                  60655
Drivetrain             115055
OptionCount                 0
MajorOptions           310481
Certified                   0
Accident Check          32305
OwnershipHistory        70501
SellingDays                 0
OriginPrice           1122205
dtype: int64

-------------------------------

##### GasMileage_city , GasMileage_highway 

In [25]:
# GasMileage_city 和 GasMileage_highway 的處理方式:
# 直接 drop 空值的資料

In [26]:
new_data2 = new_data2.dropna(axis=0, subset=['GasMileage_city'])

In [27]:
new_data2.isnull().sum()

MakeId                     0
Title                      0
BodyStyle                  0
Rental                     0
Location                   0
GasMileage_city            0
GasMileage_highway         0
Price                  90369
Mileage                24260
Transmission           12707
FuelType                   0
ExteriorColor          39515
InteriorColor         301309
DoorsNum              873375
MaximumSeating             7
Engine                  3292
Drivetrain                 0
OptionCount                0
MajorOptions          216202
Certified                  0
Accident Check         17444
OwnershipHistory       46999
SellingDays                0
OriginPrice           883588
dtype: int64

----------------------------

##### Transmission

In [28]:
new_data3 = pd.DataFrame()

In [29]:
type(new_data3)

pandas.core.frame.DataFrame

In [30]:
for id in new_data2['MakeId'].unique():
    part = new_data2[new_data2['MakeId'] == id]
    most = part.groupby(by='Transmission').size().sort_values(ascending=False).index[0]
    part['Transmission'] = part['Transmission'].fillna(value=most)
    new_data3 = new_data3.append(part)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
new_data3.isnull().sum()

MakeId                     0
Title                      0
BodyStyle                  0
Rental                     0
Location                   0
GasMileage_city            0
GasMileage_highway         0
Price                  90369
Mileage                24260
Transmission               0
FuelType                   0
ExteriorColor          39515
InteriorColor         301309
DoorsNum              873375
MaximumSeating             7
Engine                  3292
Drivetrain                 0
OptionCount                0
MajorOptions          216202
Certified                  0
Accident Check         17444
OwnershipHistory       46999
SellingDays                0
OriginPrice           883588
dtype: int64

-------------------------------------

##### MaximumSeating

In [32]:
new_data3[new_data3['MaximumSeating'].isnull()]

,MakeId,Title,BodyStyle,Rental,Location,GasMileage_city,GasMileage_highway,Price,Mileage,Transmission,FuelType,ExteriorColor,InteriorColor,DoorsNum,MaximumSeating,Engine,Drivetrain,OptionCount,MajorOptions,Certified,Accident Check,OwnershipHistory,SellingDays,OriginPrice
952553,m22,2016 Cadillac XTS Pro Coachbuilder Limousine F...,sedan,False,"Irving, TX",15 MPG City,21 MPG Highway,62995.0,40228.0,6-Speed Automatic,Gasoline,Crystal White,Shale With Cocoa Accents,NaN,NaN,V6,Front-Wheel Drive,0,NaN,0,0.0,NaN,135,65779.0
953166,m22,2015 Cadillac XTS Pro Coachbuilder Limousine F...,sedan,True,"Chicago, IL",14 MPG City,21 MPG Highway,10995.0,175558.0,6-Speed Automatic,Gasoline,Black,Black,NaN,NaN,V6,Front-Wheel Drive,0,NaN,0,0.0,5.0,126,10995.0
953872,m22,2015 Cadillac XTS Pro Coachbuilder Limousine F...,sedan,False,"Butler, PA",14 MPG City,21 MPG Highway,25995.0,45980.0,Automatic,Gasoline,Black Raven,Jet Black,NaN,NaN,V6,Front-Wheel Drive,0,NaN,0,0.0,1.0,118,NaN
960552,m22,2014 Cadillac XTS Pro Coachbuilder Limousine F...,sedan,True,"Baton Rouge, LA",15 MPG City,21 MPG Highway,13495.0,103492.0,6-Speed Automatic,Gasoline,Black,Black,NaN,NaN,V6,Front-Wheel Drive,6,"Bluetooth,Leather Seats,Premium Wheels,Backup ...",0,1.0,1.0,64,NaN
969885,m22,2015 Cadillac XTS Pro Coachbuilder Limousine FWD,sedan,False,"Cockeysville, MD",14 MPG City,21 MPG Highway,NaN,47003.0,6-Speed Automatic,Gasoline,Black Raven,jet black,NaN,NaN,V6,Front-Wheel Drive,0,NaN,0,2.0,1.0,29,NaN
979235,m22,2014 Cadillac XTS Pro Coachbuilder Limousine FWD,sedan,False,"Ewing, NJ",15 MPG City,21 MPG Highway,NaN,NaN,6-Speed Automatic,Gasoline,Black Diamond,Jet Black,NaN,NaN,V6,Front-Wheel Drive,1,Leather Seats,0,1.0,2.0,9,NaN
1964926,m55,"2015 Volkswagen Passat SE w/ Convenience - $8,495",sedan,True,"Boise, ID",24 MPG City,36 MPG Highway,8495.0,76073.0,6-Speed Automatic,Gasoline,Red,NaN,NaN,NaN,I4,Front-Wheel Drive,0,NaN,0,2.0,3.0,96,7695.0


In [33]:
m22 = new_data3[new_data3['MakeId'] == 'm22']

In [34]:
m22.groupby('MaximumSeating').size().sort_values(ascending=False)

MaximumSeating
5 seats    23735
8 seats     6910
4 seats     1172
6 seats      953
7 seats      425
2 seats      171
dtype: int64

In [35]:
new_data3[new_data3['MakeId'] == 'm55'].groupby('MaximumSeating').size().sort_values(ascending=False)

MaximumSeating
5 seats    39637
7 seats     4436
4 seats     4200
6 seats        3
dtype: int64

In [36]:
new_data3.MaximumSeating.fillna('5 seats', inplace=True)

In [37]:
new_data3.isnull().sum()

MakeId                     0
Title                      0
BodyStyle                  0
Rental                     0
Location                   0
GasMileage_city            0
GasMileage_highway         0
Price                  90369
Mileage                24260
Transmission               0
FuelType                   0
ExteriorColor          39515
InteriorColor         301309
DoorsNum              873375
MaximumSeating             0
Engine                  3292
Drivetrain                 0
OptionCount                0
MajorOptions          216202
Certified                  0
Accident Check         17444
OwnershipHistory       46999
SellingDays                0
OriginPrice           883588
dtype: int64

------------------------------------

##### DoorsNum

In [38]:
new_data4 = pd.DataFrame()

In [39]:
for id in new_data3['MakeId'].unique():
    if id != 'm203':
        part = new_data3[new_data3['MakeId'] == id]
        most = part.groupby(by='DoorsNum').size().sort_values(ascending=False).index[0]
        part['DoorsNum'] = part['DoorsNum'].fillna(value=most)
        new_data4 = new_data4.append(part)
    else:
        part = new_data3[new_data3['MakeId'] == id]
        part['DoorsNum'] = part['DoorsNum'].fillna(value='4 doors')
        new_data4 = new_data4.append(part)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [40]:
new_data4.isnull().sum()

MakeId                     0
Title                      0
BodyStyle                  0
Rental                     0
Location                   0
GasMileage_city            0
GasMileage_highway         0
Price                  90369
Mileage                24260
Transmission               0
FuelType                   0
ExteriorColor          39515
InteriorColor         301309
DoorsNum                   0
MaximumSeating             0
Engine                  3292
Drivetrain                 0
OptionCount                0
MajorOptions          216202
Certified                  0
Accident Check         17444
OwnershipHistory       46999
SellingDays                0
OriginPrice           883588
dtype: int64

##### Engine

In [41]:
# for id in new_data4['MakeId'].unique():
#     print(id)
#     part = new_data4[new_data4['MakeId'] == id]
#     print(part.groupby('Engine').size().sort_values(ascending=False))

##### OriginPrice

In [50]:
test_data = new_data4.dropna(subset=['Price','OriginPrice'])
# test_data 就是之後要用來訓練的資料 Price 和 OriginPrice 都有值

In [51]:
len(test_data)

1017565

In [52]:
test_data[test_data['Price'] == test_data['OriginPrice']]
# 去掉所有空值後,發現價格完全沒有變動過的資料只有13908筆, 表示價格變動的滿劇烈的

,MakeId,Title,BodyStyle,Rental,Location,GasMileage_city,GasMileage_highway,Price,Mileage,Transmission,FuelType,ExteriorColor,InteriorColor,DoorsNum,MaximumSeating,Engine,Drivetrain,OptionCount,MajorOptions,Certified,Accident Check,OwnershipHistory,SellingDays,OriginPrice
51,m1,"2010 Chevrolet Malibu LS FWD - $10,995",sedan,False,"Schuylkill Haven, PA",22 MPG City,30 MPG Highway,10995.0,28988.0,4-Speed Automatic,Gasoline,Gold Mist Metallic,NaN,4 doors,5 seats,I4,Front-Wheel Drive,0,NaN,0,0.0,1.0,1624,10995.0
89,m1,2016 Chevrolet Corvette Stingray 2LT Convertib...,convertible,False,"North Dartmouth, MA",16 MPG City,29 MPG Highway,57250.0,105.0,8-Speed Automatic,Gasoline,ARCTIC WHITE,KALAHARI,2 doors,2 seats,V8,Rear-Wheel Drive,5,"Leather Seats,Navigation System,Bluetooth,Back...",0,0.0,NaN,1537,57250.0
212,m1,2016 Chevrolet Corvette Stingray Z51 2LT Conve...,convertible,False,"North Dartmouth, MA",16 MPG City,29 MPG Highway,60815.0,103.0,8-Speed Automatic,Gasoline,ARCTIC WHITE,KALAHARI,2 doors,2 seats,V8,Rear-Wheel Drive,5,"Leather Seats,Navigation System,Bluetooth,Back...",0,0.0,NaN,1365,60815.0
256,m1,"2015 Chevrolet SS RWD - $49,000",sedan,False,"Bridgeport, CT",14 MPG City,21 MPG Highway,49000.0,15100.0,6-Speed Automatic,Gasoline,Black,Black,4 doors,5 seats,V8,Rear-Wheel Drive,2,"Navigation System,Bluetooth",0,0.0,1.0,1316,49000.0
282,m1,"2009 Chevrolet Impala 1LT FWD - $7,995",sedan,True,"Houston, TX",19 MPG City,29 MPG Highway,7995.0,93395.0,4-Speed Automatic,Flex Fuel Vehicle,White,Black Cloth,4 doors,6 seats,V6 Flex Fuel Vehicle,Front-Wheel Drive,1,Alloy Wheels,0,0.0,4.0,1263,7995.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373772,m98,"2013 FIAT 500 Abarth Convertible - $15,912",convertible,False,"East Hanover, NJ",28 MPG City,34 MPG Highway,15912.0,13222.0,5-Speed Manual,Gasoline,Nero,Black,2 doors,4 seats,I4,Front-Wheel Drive,1,Bluetooth,0,0.0,2.0,30,15912.0
2373996,m98,"2012 FIAT 500 Pop - $5,995",hatchback,False,"Wilmington, CA",30 MPG City,38 MPG Highway,5995.0,62538.0,Manual,Gasoline,Bianco Perla (Pearl White Tri-Coat),White (Ivory Interior),2 doors,4 seats,I4,Front-Wheel Drive,3,"Audio Package,Steel Wheels,Bluetooth",0,0.0,3.0,25,5995.0
2374126,m98,"2015 FIAT 500 Pop - $5,989",hatchback,False,"Avondale, AZ",31 MPG City,40 MPG Highway,5989.0,46762.0,5-Speed Manual,Gasoline,Blue Metallic,Gray,2 doors,4 seats,I4,Front-Wheel Drive,2,"Steel Wheels,Quick Order Package",0,0.0,1.0,21,5989.0
2374214,m98,"2012 FIAT 500 Lounge Convertible - $8,500",convertible,True,"Silver Spring, MD",27 MPG City,32 MPG Highway,8500.0,71859.0,6-Speed Automatic,Gasoline,Bianco (White),Avorio (Ivory),2 doors,4 seats,I4,Front-Wheel Drive,4,"Leather Seats,Alloy Wheels,Bluetooth,Quick Ord...",0,0.0,3.0,19,8500.0


In [53]:
train_data = test_data.iloc[:,[7,22,23]]

In [54]:
train_data['SellingDays'] = train_data['SellingDays'].astype('int32')

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [55]:
train_data.dtypes

Price          float64
SellingDays      int32
OriginPrice    float64
dtype: object

In [56]:
train_data.corr()

,Price,SellingDays,OriginPrice
Price,1.000000,0.007153,0.008534
SellingDays,0.007153,1.000000,-0.000358
OriginPrice,0.008534,-0.000358,1.000000


In [57]:
from sklearn.ensemble import RandomForestRegressor

In [58]:
forest = RandomForestRegressor(n_estimators=10,
                              criterion='mse',
                              random_state=1,
                              n_jobs=-1)

In [59]:
y_train = train_data.iloc[:,2]

In [60]:
X_train = train_data.iloc[:,[0,1]]

In [61]:
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                      oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [62]:
need_pred = new_data4.dropna(subset=['Price']).iloc[:,[7,22,23]]

In [63]:
need_pred = need_pred[need_pred['SellingDays'] != 0]

In [65]:
need_pred.isnull().sum()
# 在 need_pred 當中 , Price 和 SellingDays都是有值的

Price               0
SellingDays         0
OriginPrice    777005
dtype: int64

In [66]:
need_pred = need_pred[need_pred['OriginPrice'].isnull()]

In [67]:
X_test = need_pred.iloc[:,[0,1]]

In [68]:
y_pred = forest.predict(X_test)

In [69]:
the_index = need_pred['OriginPrice'].index

In [70]:
len(the_index) == len(y_pred)

True

In [71]:
type(y_pred)

numpy.ndarray

In [72]:
y_pred

array([ 8252.9 ,  9965.  , 13583.4 , ..., 12442.7 ,  7375.43, 17254.  ])

In [73]:
need_pred.loc[the_index,'OriginPrice'] = y_pred

c:\program files\python37\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [74]:
need_pred

,Price,SellingDays,OriginPrice
5,4890.0,1809,8252.90
7,6550.0,1809,9965.00
9,9220.0,1809,13583.40
13,3140.0,1809,9935.70
14,16995.0,1799,21272.90
...,...,...,...
2375293,16598.0,1,16887.80
2375295,8988.0,1,10993.80
2375297,11988.0,1,12442.70
2375298,7500.0,1,7375.43


In [75]:
new_data4.loc[the_index,'OriginPrice'] = y_pred

In [76]:
new_data4
# 因為有些資料被 drop掉了 (Price是空值的那些) , 所以在補 OriginPrice的時候, 不會全部都補到

,MakeId,Title,BodyStyle,Rental,Location,GasMileage_city,GasMileage_highway,Price,Mileage,Transmission,FuelType,ExteriorColor,InteriorColor,DoorsNum,MaximumSeating,Engine,Drivetrain,OptionCount,MajorOptions,Certified,Accident Check,OwnershipHistory,SellingDays,OriginPrice
3,m1,2014 Chevrolet Corvette Stingray 3LT Coupe RWD...,coupe,False,"Lewistown, MT",17 MPG City,29 MPG Highway,49995.0,2500.0,7-Speed Manual,Gasoline,Red,Black,2 doors,2 seats,V8,Rear-Wheel Drive,5,"Bluetooth,Backup Camera,Aluminum Wheels,Naviga...",0,0.0,1.0,1832,62900.0
4,m1,"2002 Chevrolet Monte Carlo SS FWD - $3,440",coupe,True,"Delavan, IL",17 MPG City,27 MPG Highway,3440.0,174000.0,Automatic,Gasoline,Black,Black,2 doors,5 seats,V6,Front-Wheel Drive,2,"Leather Seats,Aluminum Wheels",0,0.0,2.0,1809,4800.0
5,m1,"2004 Chevrolet Monte Carlo LS FWD - $4,890",coupe,True,"Delavan, IL",19 MPG City,29 MPG Highway,4890.0,115500.0,Automatic,Gasoline,Medium Gray Metallic,Medium Gray,2 doors,5 seats,V6,Front-Wheel Drive,1,Steel Wheels,0,0.0,3.0,1809,8252.9
7,m1,2001 Chevrolet Silverado 1500 LT Extended Cab ...,pickup_truck,True,"Delavan, IL",14 MPG City,18 MPG Highway,6550.0,188583.0,Automatic,Gasoline,Med Charcoal Gray Met,Graphite,4 doors,5 seats,V8,Rear-Wheel Drive,3,"Leather Seats,Suspension Package,Aluminum Wheels",0,0.0,1.0,1809,9965.0
8,m1,"2009 Chevrolet Equinox 2LT FWD - $9,960",crossover,True,"Delavan, IL",17 MPG City,24 MPG Highway,9960.0,80124.0,Automatic,Gasoline,Black Granite Metallic,NaN,4 doors,5 seats,V6,Front-Wheel Drive,4,"Leather Seats,Chrome Wheels,Bluetooth,Remote S...",0,0.0,2.0,1809,12740.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2375341,m98,"2014 FIAT 500 Pop - $11,980",hatchback,True,"Clearfield, PA",27 MPG City,34 MPG Highway,11980.0,9493.0,6-Speed Automatic,Gasoline,Blue,Black,2 doors,4 seats,I4,Front-Wheel Drive,1,Bluetooth,0,0.0,1.0,1920,12980.0
2375346,m98,2013 FIAT 500 Sport,hatchback,False,"Eynon, PA",31 MPG City,40 MPG Highway,NaN,NaN,5-Speed Manual,Gasoline,Black,Black,2 doors,4 seats,I4,Front-Wheel Drive,0,NaN,0,0.0,3.0,1911,NaN
2375347,m98,2012 FIAT 500 Lounge,hatchback,False,"Eynon, PA",27 MPG City,34 MPG Highway,NaN,NaN,6-Speed Automatic,Gasoline,Argento (Silver),Black,2 doors,4 seats,I4,Front-Wheel Drive,5,"Leather Seats,Sunroof/Moonroof,Alloy Wheels,Bl...",0,0.0,1.0,1911,NaN
2375348,m98,2013 FIAT 500 Lounge,hatchback,False,"Eynon, PA",31 MPG City,40 MPG Highway,NaN,NaN,5-Speed Manual,Gasoline,Rosso (Red),Avorio (Ivory),2 doors,4 seats,I4,Front-Wheel Drive,0,NaN,0,0.0,1.0,1911,NaN


In [77]:
new_data4.isnull().sum()

MakeId                     0
Title                      0
BodyStyle                  0
Rental                     0
Location                   0
GasMileage_city            0
GasMileage_highway         0
Price                  90369
Mileage                24260
Transmission               0
FuelType                   0
ExteriorColor          39515
InteriorColor         301309
DoorsNum                   0
MaximumSeating             0
Engine                  3292
Drivetrain                 0
OptionCount                0
MajorOptions          216202
Certified                  0
Accident Check         17444
OwnershipHistory       46999
SellingDays                0
OriginPrice           106583
dtype: int64

In [ ]:
# 上面的補法還是有問題 Selling days 是 0 天的那些 應該要拿掉再做補值
# 已經修正過了 Selling days 如果是 0 天 ,就不用這個方式補值

In [78]:
new_data4['OwnershipHistory'].unique()

array([ 1.,  2.,  3.,  4.,  7.,  5.,  6., nan,  8.,  9., 15., 14., 11.,
       10., 12., 17., 13., 16.])

In [79]:
new_data4['Accident Check'].unique()

array([ 0.,  2.,  1., nan,  3.,  4.,  5.,  6.,  7.,  8.])

In [82]:
new_data4.to_csv('C:\\machine_learning\\20200203_final_data.csv', index=False)